In [26]:
#collecting data
import cv2
import mediapipe as mp
import pandas as pd

mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
video = cv2.VideoCapture(0)

data = []
label = 'Z'  # Change this as needed

# Reads a frame from the webcam, flips it (for mirror view), and sends it to MediaPipe for hand detection.


while True:
    suc,img = video.read()
    img_rgb = cv2.cvtColor(cv2.flip(img, 1), cv2.COLOR_BGR2RGB)
    results = hands.process(img_rgb)


# If a hand is detected:

# Collects 21 landmarks (x, y, z for each) = 63 values.

# Adds the current label.

# Stores the data in the data list.

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            landmarks = []
            for lm in hand_landmarks.landmark:
                landmarks.extend([lm.x, lm.y, lm.z])
            landmarks.append(label)
            data.append(landmarks)

    cv2.imshow('Image', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()

import os
import pandas as pd
df = pd.DataFrame(data)
csv_file = 'asl_data.csv'

# Append vertically
if os.path.exists(csv_file):
    df.to_csv(csv_file, mode='a', index=False, header=False)    # append rows, no header
else:
    df.to_csv(csv_file, index=False, header=True)               # create file with header


In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

df = pd.read_csv('asl_data.csv')
X = df.drop(df.columns[-1], axis=1)
y = df[df.columns[-1]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
model = KNeighborsClassifier(n_neighbors=5)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           A       0.93      0.97      0.95        29
           B       0.98      1.00      0.99        48
           C       1.00      1.00      1.00        32
           D       1.00      1.00      1.00        72
           E       0.97      0.90      0.93        31
           F       1.00      0.94      0.97        16
           G       0.95      1.00      0.97        18
           H       1.00      0.94      0.97        35
           I       0.95      1.00      0.97        18
           J       1.00      1.00      1.00        18
           K       0.83      1.00      0.91        10
           L       1.00      0.92      0.96        25
           M       1.00      1.00      1.00        26
           N       0.91      1.00      0.95        10
           O       1.00      1.00      1.00        23
           P       1.00      1.00      1.00        24
           Q       1.00      1.00      1.00        35
           R       0.96    

In [31]:
import cv2
import mediapipe as mp
import time
import pyttsx3

mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
video = cv2.VideoCapture(0)

sentence = ""
current_letter = ""
engine = pyttsx3.init()

while True:
    success, img = video.read()
    img = cv2.flip(img, 1)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(img_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            landmarks = []
            for lm in hand_landmarks.landmark:
                landmarks.extend([lm.x, lm.y, lm.z])

            prediction = model.predict([landmarks])
            current_letter = prediction[0]  # Store current prediction

    # Display current letter and formed sentence
    cv2.putText(img, f'system detecting_letter: {current_letter}', (10, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    cv2.putText(img, f'Sentence: {sentence}', (10, 80),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("ASL Sentence Builder", img)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord(' '):  # Spacebar confirms adding the current letter
        sentence += current_letter
    elif key == ord('c'):  # 'c' to clear the sentence
        sentence = ""
    elif key == ord('b'):  # 'b' to backspace one letter
        sentence = sentence[:-1]
    elif key == ord('s'):  # 's' to insert space
        sentence += " "


    if key == ord('o'):
        print("Confirmed Sentence:", sentence)
        engine.say(sentence)
        engine.runAndWait()
        sentence = ""  # Reset sentence

    if key == ord('q'):
        break


video.release()
cv2.destroyAllWindows()


c:\Users\neham\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
c:\Users\neham\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
c:\Users\neham\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
c:\Users\neham\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
c:\Users\neham\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\uti

Confirmed Sentence: AAAAAA
